In [15]:
from PIL import Image
import os
import numpy as np

In [18]:
path = os.path.join(os.getcwd().split(r'\notebooks')[0], r'data\HealthyHuman\2022-07-06_T_autopsy-bloc_BF_FR_M1_1\annotation\merged.png')
img = np.array(Image.open(path))

In [19]:
GM_WM_colored = np.zeros((img.shape[0], img.shape[1], 3))
TCC_colored = np.zeros((img.shape[0], img.shape[1], 3))

for idx, x in enumerate(img):
    for idy, y in enumerate(x):
        if y != 0:
            if y != 0:
                TCC_colored[idx, idy] = [0, 255, 0]
                
                if y == 128:
                    GM_WM_colored[idx, idy] = [153, 77, 0]
                elif y == 255:
                    GM_WM_colored[idx, idy] = [153, 153, 0]

In [20]:
Image.fromarray(TCC_colored.astype(np.uint8)).save(path.replace('merged', 'TCC_colored'))
Image.fromarray(GM_WM_colored.astype(np.uint8)).save(path.replace('merged', 'GM_WM_colored'))

In [5]:
from PIL import Image
import os
import numpy as np

WM_path = os.path.join(os.getcwd().split(r'\notebooks')[0], r'data\TumorMeasurements\63\2022-11-08_T_HORAO-63-BF_FR_S1_1\annotation\WM_1.tif')
BG_path = os.path.join(os.getcwd().split(r'\notebooks')[0], r'data\TumorMeasurements\63\2022-11-08_T_HORAO-63-BF_FR_S1_1\annotation\BG_1.tif')
WM = np.array(Image.open(WM_path))
BG = np.array(Image.open(BG_path))

In [6]:
GM_WM_colored = np.zeros((WM.shape[0], WM.shape[1], 3))

In [7]:
for idx, x in enumerate(WM):
    for idy, y in enumerate(x):
        if y != 0:
            GM_WM_colored[idx, idy] = [153, 153, 0]
        elif BG[idx, idy] != 0:
            pass
        else:
            GM_WM_colored[idx, idy] = [153, 77, 0]

In [8]:
Image.fromarray(GM_WM_colored.astype(np.uint8)).save(WM_path.replace('WM_1.tif', 'GM_WM_colored.png'))

In [10]:
from PIL import Image
import os
import numpy as np
import cv2

path = os.path.join(os.getcwd().split(r'\notebooks')[0], r'data\HealthyHuman\2022-07-06_T_autopsy-bloc_BF_FR_M1_1\annotation\merged.png')
img = np.array(Image.open(path))

In [11]:
def remove_background(FG):
    (thresh, FG) = cv2.threshold(FG.astype(np.uint8), 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    kernel = np.ones((5, 5), np.uint8) 
    FG = cv2.erode(FG, kernel)
    FG_contour = cv2.Canny(FG, 30, 200) 
    FG_contour = cv2.dilate(FG_contour, kernel)
    return FG, FG_contour

In [12]:
path_polarimetry = r'C:\Users\romai\Documents\PathologyPaper\data\HealthyHuman\2022-07-06_T_autopsy-bloc_BF_FR_M1_1\polarimetry\550nm'
paths = [os.path.join(path_polarimetry, 'Intensity_img.png'), os.path.join(path_polarimetry, 'Azimuth of optical axis_img.png'),
            os.path.join(path_polarimetry, 'Depolarization_img.png'), os.path.join(path_polarimetry, 'Linear retardance_img.png')]

FG, FG_contour = remove_background(img != 0 )

for path in paths:
    img = np.array(Image.open(path))
    labels = mask_labels(img, FG, FG_contour, polarimetry = True)
    Image.fromarray(labels).save(path.replace('img', 'img_masked'))

In [9]:
def mask_labels(labels, mask, mask_contour = None, polarimetry = False):
    masked = np.zeros(labels.shape)
    for idx, x in enumerate(labels):
        for idy, y in enumerate(x):
            if mask[idx, idy] == 0:
                if polarimetry:
                    masked[idx, idy] = [0, 0, 0, 255]
                else:
                    pass
            else:
                masked[idx, idy] = y
                
            if polarimetry:
                if mask_contour[idx, idy] != 0:
                    masked[idx, idy] = [255, 255, 255, 255]
    return masked.astype(np.uint8)